In [2]:
import os 
from src.preprocessing_data import all_preprocessing

data_name = 'data.xlsx'
feature_column = ['Правило взаимодействия (обр.)', 'Исходники (обр.)']
new_data_name = 'encoded.csv'
df, features, tok = all_preprocessing(
    os.path.join('data', data_name), 
    feature_column, 
    os.path.join('data', new_data_name)
    ) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Анечка\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Анечка\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
a:\Programs\Python_3_8_5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 10094/10094 [00:00<00:00, 12400.51it/s]


In [3]:
df['labels']

0        [0, 0, 0, 0, 0, 1, 0, 0]
1        [0, 0, 0, 0, 0, 1, 0, 0]
2        [0, 0, 0, 0, 0, 1, 0, 0]
3        [0, 0, 0, 0, 0, 0, 0, 1]
4        [0, 1, 0, 0, 0, 0, 0, 0]
                   ...           
10089    [0, 0, 1, 0, 0, 0, 0, 0]
10090    [0, 0, 1, 0, 0, 0, 0, 0]
10091    [0, 0, 1, 0, 0, 0, 0, 0]
10092    [0, 0, 1, 0, 0, 0, 0, 0]
10093    [0, 0, 1, 0, 0, 0, 0, 0]
Name: labels, Length: 10094, dtype: object

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(df.labels))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
import numpy as np

batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [7]:
import torch 
import pandas as pd 

def get_tensor(series):
    return torch.tensor(np.array(series))


In [46]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=df, batch_size=batch_size, shuffle=False)
eval_loader = DataLoader(dataset=df, batch_size=batch_size, shuffle=False)

# train = data_utils.TensorDataset(df[df.columns[0:3]], df[df.columns[3]])
# train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False)

# tr = IterableDataset()

# tr = df.map(lambda x: x)
# ev = df.map(lambda x: x)

trainer = Trainer(
    model,
    args,
    train_dataset=df,
    eval_dataset=df,
    tokenizer=tok,
    compute_metrics=compute_metrics
)

In [47]:
trainer.train()

a:\Programs\Python_3_8_5\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



KeyError: 2476